In [1]:
import os
import time
import datetime

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup


In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
path = 'C:/Users/wjtay/Documents/GitHub/medicalbot/gpt_models/model_save_3'
model =  GPT2LMHeadModel.from_pretrained('C:/Users/wjtay/Documents/GitHub/medicalbot/gpt_models/model_save_3_lasseOnly_30')
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [6]:
model.eval()

q1 = "throat a bit sore and want to get a good imune booster, especially in light of the virus. please advise. have not been in contact with nyone with the virus."
q2 = "i need a list of foods that i can eat and list of foods to avoid with pancreatitis"
q3 = "i made a mistake and i bought 9 to 24 months formula for my newborn. can he drink it?"
q4 = "i think i have bone cancer. what should i do?"
q5 = "i have covid-19 what should i do?"

prompt = f"<|startoftext|><|question|>{q1}<|answer|>"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
# generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257,    27,    91, 25652,    91,    29, 26110,   265,   257,  1643,
         19597,   290,   765,   284,   651,   257,   922,   545,  1726, 30430,
            11,  2592,   287,  1657,   286,   262,  9471,    13,  3387, 18595,
            13,   423,   407,   587,   287,  2800,   351,   299,    88,   505,
           351,   262,  9471, 29847,    91, 41484,    91,    29]])
0: <|question|>throat a bit sore and want to get a good imune booster, especially in light of the virus. please advise. have not been in contact with nyone with the virus.<|answer|>most helpful answer that would be fine. the virus is called h1n1 so you'd be well-advised to do this. the sooner you know for sure the better. as you know hepatitis b and hepatitis c can be transmitted very easily and most cases of the virus are in the liver. why in the world would you want a booster? it's not likely the virus is transmitted that way. it's more likely you'll spend your energy and time looking for an effective vaccin

In [ ]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
model2 = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model2.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [14]:
model2.eval()

prompt = "<|startoftext|>my son has add and mild autism. he has been successfully on concerta for 6+ years. can you help with his weight loss?"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
# generated = generated.to(device)

print(generated)

sample_outputs = model2.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 300,
                                top_p=0.95, 
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257,  1820,  3367,   468,   751,   290, 11607, 15998,    13,   339,
           468,   587,  7675,   319, 10010,    64,   329,   718,    10,   812,
            13,   460,   345,  1037,   351,   465,  3463,  2994,    30]])
0: my son has add and mild autism. he has been successfully on concerta for 6+ years. can you help with his weight loss?


1: my son has add and mild autism. he has been successfully on concerta for 6+ years. can you help with his weight loss?


2: my son has add and mild autism. he has been successfully on concerta for 6+ years. can you help with his weight loss?




In [8]:

import pandas as pd
import pickle
# with open('filename.pickle', 'wb') as handle:
#     pickle.dump(your_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load data (deserialize)
# with open('training_data/MBERT_Lasse_FAISS_embeddings.pkl', 'rb') as handle:
with open('gpu_data/FAISS_500_Med_BERT_Lasse_Only.pkl', 'rb') as handle:
    unserialized_data = pickle.load(handle)

training_df1 = pd.DataFrame(unserialized_data)
print(training_df1.shape)
display(training_df1.head())

# training_df2 = pd.read_csv('translated_1_2_3.csv')
# print(training_df2.shape)


# with open('gpu_data/BERT_Lasse_Only_FAISS_embeddings.pkl', 'rb') as handle:
#     unserialized_data = pickle.load(handle)

# training_df2 = pd.DataFrame(unserialized_data)
# print(training_df2.shape)

(31822, 4)


,question,answer,question_embedding,answer_embedding
0,my 5 1/2-year-old son displays adhd symptoms f...,adhd and bipolar mood disorder (bmd) can coexi...,"[-0.11883694, 0.08731164, -0.044894945, -0.067...","[-0.07025841, 0.007099037, 0.042368226, -0.115..."
1,my son has add and mild autism. he has been su...,stimulants in general tend to decrease appetit...,"[-0.07242921, -0.03825957, 0.04104553, -0.0251...","[-0.086425886, 0.14200483, -0.04221771, -0.036..."
2,my son is 13 and is depressed. he has been tak...,while any of the stimulant medications can inc...,"[-0.1124784, 0.08052201, 0.01542189, 0.1301103...","[-0.030669264, 0.13411666, -0.07706964, -0.018..."
3,my 17-year-old has stopped taking concerta aft...,seventy percent of teens diagnosed when they a...,"[-0.06017951, 0.037160657, -0.090312295, 0.004...","[-0.05405549, 0.17740452, -0.06584634, -0.0168..."
4,i've been taking respa-ar for allergies. i can...,try claritin-d which is located behind the pha...,"[-0.12791637, 0.18242685, 0.048871003, 0.09654...","[-0.05976576, 0.19645679, 0.04084024, 0.057927..."


In [7]:
set1 = set()
for idx, row in training_df1.iterrows():
    item = tuple([row['question'], row['answer']])
    set1.add(item)

set2 = set()
for idx, row in training_df2.iterrows():
    item = tuple([row['question'], row['answer']])
    set2.add(item)

print(f'set1: {len(set1)}')
print(f'set2: {len(set2)}')

diff = set1.symmetric_difference(set2)
print(f'diff {len(diff)}')

set1: 31008
set2: 31105
diff 2007


In [5]:
print(training_df.iloc[-41]["question"])
print(training_df.iloc[-42]["question"])
print(training_df.iloc[-43]["question"])
print(training_df.iloc[-44]["question"])

why am i bleeding after taking contraceptive pill?
should i take some medicine to stop heavy bleeding after an i-pill?
can antibiotics interfere with the depo-provera birth control shot the way they do with oral contraceptives?
will antibiotics reduce the effect of oral contraceptive pills?


In [9]:
display(training_df[training_df["question"] == "should i take some medicine to stop heavy bleeding after an i-pill?"])

,question,answer,question_embedding,answer_embedding
29531,should i take some medicine to stop heavy blee...,the bleeding could be due to withdrawal effect...,"[-0.4998133, 0.21075934, -0.1710357, 0.1950991...","[-0.12213257, 0.26806858, 0.04654521, 0.139121..."
31743,should i take some medicine to stop heavy blee...,the extra bleeding could be due to the pill. h...,"[-0.4998133, 0.21075934, -0.1710357, 0.1950991...","[-0.2732135, -0.006451058, 0.19289777, 0.24179..."
31746,should i take some medicine to stop heavy blee...,the extra bleeding could be due to the pill. h...,"[-0.4232357, 0.27518612, -0.1223748, 0.1546501...","[-0.12213257, 0.26806858, 0.04654521, 0.139121..."


In [21]:
print(training_df.iloc[29531]["question_embedding"] == training_df.iloc[31743]["question_embedding"])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [19]:
print(training_df.iloc[29531]["question"])

should i take some medicine to stop heavy bleeding after an i-pill?
